# ДЗ

Используя  RandomForestClassifier и еще 2 метода МО предсказать будущий  ABC класс на основе любых фичей


In [1]:
import pandas as pd
import numpy as np

In [2]:
df_dataset = pd.read_excel('data/DataSet_w_NA.xlsx', sheet_name = "Испорченные факты")
df_dataset = df_dataset.dropna()

In [3]:
df_dataset.head()

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3


In [4]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9993 entries, 0 to 10292
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Номер заказа            9993 non-null   object 
 1   Возврат                 9993 non-null   object 
 2   Дата заказа             9993 non-null   object 
 3   Дата отправки           9993 non-null   object 
 4   Режим отправки          9993 non-null   object 
 5   Факты.Контрагент ID     9993 non-null   object 
 6   Факты.Товар ID          9993 non-null   object 
 7   Продажи, руб            9993 non-null   float64
 8   Продажи, шт             9993 non-null   float64
 9   Скидка                  9993 non-null   float64
 10  Маржинальная прибыль    9993 non-null   float64
 11  Повторение заказа       9993 non-null   int64  
 12  Повторение контрагента  9993 non-null   int64  
 13  Повторение товара       9993 non-null   int64  
dtypes: float64(4), int64(3), object(7)
memo

In [5]:
df = pd.pivot_table(df_dataset, values=['Продажи, руб', 'Продажи, шт','Повторение заказа','Маржинальная прибыль','Повторение товара'], index=["Факты.Товар ID"],
                     aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': np.sum,
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum
                               })

In [6]:
df.head()

Маржинальная прибыль Повторение заказа Продажи, руб  \
                                sum               sum          sum   
Факты.Товар ID                                                       
PROD0000001               -28160.70                 6     631978.5   
PROD0000002               963721.00                32    5318764.0   
PROD0000003                32200.55                18      76755.0   
PROD0000004              -266380.65                17    3621384.0   
PROD0000005                31874.40                18     150984.0   

               Продажи, шт        
                    median   sum  
Факты.Товар ID                    
PROD0000001            2.5  12.0  
PROD0000002            3.5  51.0  
PROD0000003            2.0  23.0  
PROD0000004            4.0  29.0  
PROD0000005            5.0  24.0

In [7]:
df.columns = df.columns.map('_'.join)
df = df.reset_index()

In [8]:
df.head()

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum"
0,PROD0000001,-28160.70,6,631978.5,2.5,12.0
1,PROD0000002,963721.00,32,5318764.0,3.5,51.0
2,PROD0000003,32200.55,18,76755.0,2.0,23.0
3,PROD0000004,-266380.65,17,3621384.0,4.0,29.0
4,PROD0000005,31874.40,18,150984.0,5.0,24.0


In [9]:
total_sale = df['Продажи, руб_sum'].sum()

In [10]:
df['Доля'] = df['Продажи, руб_sum']/total_sale * 100

In [11]:
df = df.sort_values(by=('Продажи, руб_sum'), ascending=False)

In [12]:
df

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля
1388,PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845
450,PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226
1411,PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602
483,PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.952170
1176,PROD0001187,1.116753e+06,33,9911739.5,3.0,37.0,0.863046
...,...,...,...,...,...,...,...
1838,PROD0001871,1.578500e+03,1,3850.0,2.0,2.0,0.000335
1854,PROD0001887,1.193400e+03,1,3536.0,2.0,2.0,0.000308
1747,PROD0001779,1.555200e+03,3,3240.0,1.0,1.0,0.000282
1675,PROD0001703,1.411200e+03,1,2880.0,2.0,2.0,0.000251


In [13]:
df = df.assign(sum_d=df['Доля'].cumsum())

In [14]:
df

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d
1388,PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845
450,PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071
1411,PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673
483,PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.952170,5.814843
1176,PROD0001187,1.116753e+06,33,9911739.5,3.0,37.0,0.863046,6.677889
...,...,...,...,...,...,...,...,...
1838,PROD0001871,1.578500e+03,1,3850.0,2.0,2.0,0.000335,99.999089
1854,PROD0001887,1.193400e+03,1,3536.0,2.0,2.0,0.000308,99.999396
1747,PROD0001779,1.555200e+03,3,3240.0,1.0,1.0,0.000282,99.999679
1675,PROD0001703,1.411200e+03,1,2880.0,2.0,2.0,0.000251,99.999929


In [15]:
df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'

In [16]:
df

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC
1388,PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A
450,PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A
1411,PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A
483,PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A
1176,PROD0001187,1.116753e+06,33,9911739.5,3.0,37.0,0.863046,6.677889,A
...,...,...,...,...,...,...,...,...,...
1838,PROD0001871,1.578500e+03,1,3850.0,2.0,2.0,0.000335,99.999089,C
1854,PROD0001887,1.193400e+03,1,3536.0,2.0,2.0,0.000308,99.999396,C
1747,PROD0001779,1.555200e+03,3,3240.0,1.0,1.0,0.000282,99.999679,C
1675,PROD0001703,1.411200e+03,1,2880.0,2.0,2.0,0.000251,99.999929,C


In [17]:
df['Стоимость, руб'] = df['Продажи, руб_sum']/df['Продажи, шт_sum']

In [18]:
df['Продажи в следующем периоде']= (df['Продажи, шт_sum'] + df['Продажи, шт_median']) *df['Стоимость, руб']

In [19]:
total_sale_next=df['Продажи в следующем периоде'].sum()

In [20]:
df['Доля_будущая']=df['Продажи в следующем периоде']/total_sale_next * 100

In [21]:
df = df.sort_values(by=('Продажи в следующем периоде'), ascending=False)

In [22]:
df = df.assign(sum_d_next=df['Доля_будущая'].cumsum())

In [23]:
df['Доля_будущая'].sum()

100.0

In [24]:
df.loc[(df['sum_d_next'] <= 80), 'ABC_next'] = 'A'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'ABC_next'] = 'B'
df.loc[(df['sum_d_next'] > 95), 'ABC_next'] = 'C'

In [25]:
df.loc[(df['ABC'] != df['ABC_next']), 'Изменение класса'] = 1
df.loc[(df['ABC'] == df['ABC_next']), 'Изменение класса'] = 0

In [26]:
df['Изменение класса'].sum()

61.0

In [27]:
df.loc[(df['sum_d'] <= 80), 'class'] = '0'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'class'] = '1'
df.loc[(df['sum_d'] > 95), 'class'] = '2'

In [28]:
df.loc[(df['sum_d_next'] <= 80), 'class_next'] = '0'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'class_next'] = '1'
df.loc[(df['sum_d_next'] > 95), 'class_next'] = '2'

In [29]:
df

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC,"Стоимость, руб",Продажи в следующем периоде,Доля_будущая,sum_d_next,ABC_next,Изменение класса,class,class_next
1388,PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A,1.539996e+06,3.695989e+07,2.691941,2.691941,A,0.0,0,0
1411,PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A,1.886540e+06,2.263848e+07,1.648854,4.340795,A,0.0,0,0
450,PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A,4.427965e+05,1.505508e+07,1.096523,5.437318,A,0.0,0,0
483,PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A,2.803920e+05,1.233725e+07,0.898572,6.335891,A,0.0,0,0
1629,PROD0001657,2.047488e+06,6,9187447.5,4.0,12.0,0.799980,9.126346,A,7.656206e+05,1.224993e+07,0.892213,7.228103,A,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,PROD0001257,1.050000e+03,12,4200.0,2.5,5.0,0.000366,99.997704,C,8.400000e+02,6.300000e+03,0.000459,99.998575,C,0.0,2,2
1108,PROD0001119,2.220000e+02,13,4884.0,1.5,6.0,0.000425,99.996103,C,8.140000e+02,6.105000e+03,0.000445,99.999020,C,0.0,2,2
801,PROD0000808,1.702000e+03,2,4048.0,2.5,5.0,0.000352,99.998056,C,8.096000e+02,6.072000e+03,0.000442,99.999462,C,0.0,2,2
1675,PROD0001703,1.411200e+03,1,2880.0,2.0,2.0,0.000251,99.999929,C,1.440000e+03,5.760000e+03,0.000420,99.999882,C,0.0,2,2


In [30]:
df.columns

Index(['Факты.Товар ID', 'Маржинальная прибыль_sum', 'Повторение заказа_sum',
       'Продажи, руб_sum', 'Продажи, шт_median', 'Продажи, шт_sum', 'Доля',
       'sum_d', 'ABC', 'Стоимость, руб', 'Продажи в следующем периоде',
       'Доля_будущая', 'sum_d_next', 'ABC_next', 'Изменение класса', 'class',
       'class_next'],
      dtype='object')

In [31]:
from sklearn.model_selection import train_test_split

X = df[['Повторение заказа_sum', 'Продажи, шт_median', 'Продажи, шт_sum', 'Доля', 'class', 'Стоимость, руб']]
y = df['class_next']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 85)

## RandomForestClassifier

In [32]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [33]:
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9767441860465116


## KNeighborsClassifier

In [34]:
from sklearn.neighbors import KNeighborsClassifier


X = df[['Повторение заказа_sum', 'Продажи, шт_median', 'Продажи, шт_sum', 'Доля', 'class', 'Стоимость, руб']]
y = df['class_next']

x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size = 0.2, random_state = 0)
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [35]:
knn = KNeighborsClassifier(n_neighbors=5)
knn_model_1 = knn.fit(x_train_1, y_train_1)
knn_model_2 = knn.fit(x_train_2, y_train_2)

In [36]:
knn_predictions_1 = knn_model_1.predict(x_test_1)
knn_predictions_2 = knn_model_2.predict(x_test_2)

In [37]:
from sklearn.metrics import accuracy_score
accuracy_1 = accuracy_score(y_test_1, knn_predictions_1)
accuracy_2 = accuracy_score(y_test_2, knn_predictions_2)

print(f'Accuracy_1: {accuracy_1}, accuracy_2: {accuracy_2}')

Accuracy_1: 0.8579088471849866, accuracy_2: 0.8284182305630027


## LogisticRegression

In [38]:
from sklearn import linear_model, model_selection

X = df[['Повторение заказа_sum', 'Продажи, шт_median', 'Продажи, шт_sum', 'Доля', 'class', 'Стоимость, руб']]
y = df['class_next']

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y, test_size = 0.2)
model = linear_model.LogisticRegression(max_iter = 10000)

In [39]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=10000)

In [40]:
Y_test_predicted = model.predict(X_test)

In [41]:
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(Y_test, Y_test_predicted))

Accuracy: 0.9758713136729222


### Найти причину/фактор низкой маржинальности

В исходных данных маржинальная прибыль во многих записях по ошибке указана с отрицательным значением, что скорее всего указывается на человеческий фактор при заполнении исходной таблицы.